In [ ]:
# Keras
import keras
from keras import regularizers
from keras_preprocessing import sequence
from keras_preprocessing.sequence import pad_sequences
from keras import Sequential, Model
from keras._tf_keras.keras.models import model_from_json
from keras._tf_keras.keras.layers import Dense, Embedding, LSTM, Input, Flatten, Dropout, Activation, \
    BatchNormalization, Conv1D, MaxPooling1D, AveragePooling1D
from keras._tf_keras.keras.utils import to_categorical
from keras._tf_keras.keras.callbacks import ModelCheckpoint
import np_utils

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



# Other
import librosa
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob
import os
import pickle
import IPython.display as ipd  # To play sound in the notebook

import sys
import warnings



In [61]:
# Filter specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="librosa.core.audio")

# ignore general warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [62]:
# let's pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv(r"C:\Users\eo19181\Documents\All -  Datasets\sorted_data_paths.csv")
ref.head()

,labels,source,path
0,female_angry,ESD - Dataset,C:\Users\eo19181\Documents\All - Datasets\ESD...
1,female_angry,ESD - Dataset,C:\Users\eo19181\Documents\All - Datasets\ESD...
2,female_angry,ESD - Dataset,C:\Users\eo19181\Documents\All - Datasets\ESD...
3,female_angry,ESD - Dataset,C:\Users\eo19181\Documents\All - Datasets\ESD...
4,female_angry,ESD - Dataset,C:\Users\eo19181\Documents\All - Datasets\ESD...


In [66]:


# Load an example audio file
filename = librosa.example()
y, sr = librosa.load(filename, sr=None)

print(f"Audio sample rate: {sr}")
print(f"Audio time series shape: {y.shape}")

TypeError: deprecated() takes 0 positional arguments but 2 were given

In [63]:
# Note this takes a couple of minutes (~10 mins) as we're iterating over 4 datasets
df = pd.DataFrame(columns=['feature'])

# loop feature extraction over the entire dataset
counter = 0
for index, path in enumerate(ref.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast', duration=2.5, sr=44100, offset=0.5)
    sample_rate = np.array(sample_rate)

    # mean as the feature. Could do min and max etc as well.
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    df.loc[counter] = [mfccs]
    counter += 1

# Check a few records to make sure its processed successfully
print(len(df))
df.head()


ModuleNotFoundError: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\eo19181\AppData\Roaming\Python\Python310\site-packages\librosa\core\audio.py, line 33, in <module>

----> resampy = lazy.load("resampy")

In [ ]:
# Now extract the mean bands to its own feature columns
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df[:5]